In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",1,0.264495,"""a""",0.736417,0.056087,0.249251,0.957716,0.918956,-0.246563
0.841471,1,"""a""",0,0.641356,"""a""",0.431618,0.370065,0.430757,0.601998,0.571744,-0.470281
0.909297,2,"""a""",0,0.121811,"""a""",0.737055,0.570686,0.241569,0.795424,0.492859,-0.080532
0.14112,3,"""a""",1,0.323139,"""a""",0.800407,0.441485,0.603531,0.701238,0.439433,-0.652727
-0.756802,4,"""a""",1,0.201596,"""a""",0.241561,0.295333,0.345977,0.950043,0.079215,-0.394072


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.query_jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.957716,-0.918956
-0.841471,-0.601998,-0.571744
-0.909297,-0.795424,-0.492859
-0.14112,-0.701238,-0.439433
0.756802,0.007673,0.839741


In [7]:
# Least Square (Linear Regression)
df.select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

y
list[f64]
"[-0.478902, -0.348628]"


In [8]:
df.select(
    pds.query_lstsq_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975
str,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.219311,0.001681,130.457907,0.0,0.216016,0.222606
"""exp(x2)""",0.1742,0.000678,256.973004,0.0,0.172872,0.175529
"""sin(x3)""",-1.743784,0.001334,-1306.884179,0.0,-1.746399,-1.741168
"""__bias__""",-0.107614,0.001498,-71.837625,0.0,-0.110551,-0.104678


In [9]:
df.lazy().select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = "y", # We can either put pl.col("y") here or just the string "y"
        add_bias=False
    )
).collect()

y
list[f64]
"[-0.478902, -0.348628]"


In [10]:
df.select(
    "dummy",
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

dummy,coeffs
str,list[f64]
"""a""","[-0.461479, -0.361538]"
"""a""","[-0.461479, -0.361538]"
"""a""","[-0.461479, -0.361538]"
"""a""","[-0.461479, -0.361538]"
"""a""","[-0.461479, -0.361538]"
…,…
"""b""","[-0.49687, -0.335531]"
"""b""","[-0.49687, -0.335531]"
"""b""","[-0.49687, -0.335531]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.query_lstsq(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.736417,0.056087,-0.246563,-0.372225,0.125662
0.431618,0.370065,-0.470281,-0.335718,-0.134563
0.737055,0.570686,-0.080532,-0.551934,0.471402
0.800407,0.441485,-0.652727,-0.53723,-0.115497
0.241561,0.295333,-0.394072,-0.218646,-0.175426


In [12]:
df.group_by("dummy").agg(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[-0.461479, -0.361538]"
"""b""","[-0.49687, -0.335531]"


In [13]:
# Lasso
df.group_by("dummy").agg(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[-0.30163, -0.18145]"
"""b""","[-0.323151, -0.163245]"


In [14]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.query_lstsq(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""a""",-0.524373
"""b""",-0.548477


In [15]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.query_rolling_lstsq(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-0.246563,0.736417,0.056087,null,null
-0.470281,0.431618,0.370065,null,null
-0.080532,0.737055,0.570686,null,null
-0.652727,0.800407,0.441485,null,null
-0.394072,0.241561,0.295333,"[-0.332253, -0.388705]",-0.195057
…,…,…,…,…
-0.43255,0.607058,0.978023,"[-1.590626, 0.656437]",-0.323591
-0.193283,0.450637,0.669879,"[-1.58207, 0.659216]",-0.271345
-0.002575,0.411743,0.388956,"[-1.194416, 0.394351]",-0.338408


In [16]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [17]:
# Only want singular values (principal values?)
df.select(
    pds.query_singular_values("a", "b", "x1")
)

a
list[f64]
"[28.984385, 28.823622, 28.761866]"


In [18]:
# Singular values + The principal components
df.select(
    pds.query_pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
28.902498,"[0.994242, -0.107155]"
28.817473,"[0.107155, 0.994242]"


In [19]:
# PC1
df.select(
    pds.query_principal_components("a", "b", k =1)
).unnest("principal_components").head()

pc1
f64
0.410568
0.094104
0.294869
0.206951
0.492922


# ML Metrics

In [20]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""a""",0.333926,0.997271,0.503155,0.506349,0.504747,0.5055,0.497931
"""b""",0.335692,1.017057,0.49497,0.491018,0.492986,0.497515,0.495462


# Str Extension

In [21]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [22]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

sen
str
"""world"""
"""hello"""
"""going"""
"""church"""
"""to"""


In [23]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

sen
str
"""go"""
"""church"""
""""""
"""world"""
"""hello"""


In [24]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [25]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [26]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [27]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [28]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [29]:
df.filter(
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [30]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [31]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
-1.351619
-0.409552
1.334281


In [32]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,-0.238282,null
null,1.134805,null
-1.351619,-1.364356,-0.106004
-0.409552,-1.254976,1.077692
1.334281,1.65677,0.885016


In [33]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""0v""",null
null,"""B""",null
-1.351619,"""nj""","""jtGF0"""
-0.409552,"""X""","""Ap"""
1.334281,"""u9JPd""","""B8"""


In [34]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
-1.351619,"""U1oim"""
-0.409552,"""DrN8Y"""
1.334281,"""cYubu"""


In [35]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,-0.096711,null,-0.096573
null,-1.51236,null,-1.51273
-1.351619,0.851758,1.225513,0.851806
-0.409552,-0.161385,1.11132,-0.16097
1.334281,-0.809283,1.055087,-0.809093


In [36]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.570004,0.915845,9
null,0.43759,1.214489,4
-1.351619,0.491054,-0.456213,1
-0.409552,0.557901,-0.699571,7
1.334281,0.871036,-0.086952,1


In [37]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.query_ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
-0.626703,0.53095,0.7236,0.696422


In [38]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.29699,0.229537,4,7
1,0.985198,0.309353,0,6
2,0.504475,0.543471,0,9
0,0.013544,0.76279,0,5
1,0.166792,0.965884,0,3


In [39]:
# In dataframe statistical tests!
df.select(
    pds.query_ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.query_chi2("category_1", "category_2").alias("chi2-test"),
    pds.query_f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{-0.457859,0.647064}","{29.724945,0.760403}","{1.105493,0.35203}"


In [40]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.query_ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.query_chi2("category_1", "category_2").alias("chi2-test"),
        pds.query_f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬──────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test               ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                  ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]            ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪══════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {-1.238147,0.215749} ┆ {32.302427,0.64516}  ┆ {0.428833,0.787904} │
│ 1         ┆ {0.926543,0.354231}  ┆ {35.920592,0.472368} ┆ {1.192677,0.31208}  │
│ 2         ┆ {-0.478681,0.632197} ┆ {40.283395,0.286381} ┆ {1.702241,0.146892} │
└───────────┴──────────────────────┴──────────────────────┴─────────────────────┘


In [41]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
606,0.1212
563,0.1126
544,0.1088
587,0.1174
552,0.1104
561,0.1122
512,0.1024
552,0.1104
523,0.1046


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [42]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [43]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        0.1, # radius 
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.187136,0.949014,0.734978,0.052896,9.734955,17
1,0.921853,0.710782,0.569983,0.182689,9.123167,13
2,0.608893,0.190907,0.199688,0.053402,9.219354,27
3,0.18617,0.831234,0.83613,0.763297,3.140051,16
4,0.074793,0.670172,0.704248,0.672902,0.972488,23


In [44]:
df.with_columns(
    pds.query_nb_cnt(
        pl.col("r"), # radius be an expression too
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.187136,0.949014,0.734978,0.052896,9.734955,3
1,0.921853,0.710782,0.569983,0.182689,9.123167,18
2,0.608893,0.190907,0.199688,0.053402,9.219354,1
3,0.18617,0.831234,0.83613,0.763297,3.140051,513
4,0.074793,0.670172,0.704248,0.672902,0.972488,455


In [45]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.187136,0.949014,0.734978,0.052896,9.734955,"[0, 1952, … 1989]"
1,0.921853,0.710782,0.569983,0.182689,9.123167,"[1, 1197, … 1038]"
2,0.608893,0.190907,0.199688,0.053402,9.219354,"[2, 345, … 1170]"
3,0.18617,0.831234,0.83613,0.763297,3.140051,"[3, 826, … 1946]"
4,0.074793,0.670172,0.704248,0.672902,0.972488,"[4, 491, … 235]"


In [46]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 1952, … 1209] ┆ 13                 │
│ 1   ┆ [1, 1197, … 484]  ┆ 8                  │
│ 2   ┆ [2, 345, … 1908]  ┆ 16                 │
│ 3   ┆ [3, 826, … 1353]  ┆ 10                 │
│ 4   ┆ [4, 491, … 1668]  ┆ 17                 │
└─────┴───────────────────┴────────────────────┘


In [47]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.187136,0.949014,0.734978,0.052896,9.734955,"[0, 1952, … 1989]","[0.0, 0.016217, … 0.056216]"
1,0.921853,0.710782,0.569983,0.182689,9.123167,"[1, 1197, … 1038]","[0.0, 0.038548, … 0.074352]"
2,0.608893,0.190907,0.199688,0.053402,9.219354,"[2, 345, … 1170]","[0.0, 0.04367, … 0.056832]"
3,0.18617,0.831234,0.83613,0.763297,3.140051,"[3, 826, … 1946]","[0.0, 0.026685, … 0.05296]"
4,0.074793,0.670172,0.704248,0.672902,0.972488,"[4, 491, … 235]","[0.0, 0.029128, … 0.061924]"


In [48]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
2,0.608893,0.190907,0.199688,0.053402,9.219354
6,0.475858,0.407239,0.40237,0.871159,7.561074
7,0.324005,0.862382,0.647597,0.370892,0.440285
8,0.254154,0.737911,0.446015,0.142062,1.325032
10,0.74715,0.287408,0.589737,0.025628,0.737981


In [49]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
25,0.551006,0.493076,0.891488,0.38246,8.419224
48,0.544779,0.504652,0.850245,0.147501,6.81913
72,0.46108,0.461825,0.985768,0.849787,6.563024
77,0.499401,0.452786,0.681992,0.341342,6.239652
170,0.520715,0.412681,0.113462,0.577659,1.262108


In [50]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
25,0.551006,0.493076,0.891488,0.38246,8.419224
48,0.544779,0.504652,0.850245,0.147501,6.81913
72,0.46108,0.461825,0.985768,0.849787,6.563024
77,0.499401,0.452786,0.681992,0.341342,6.239652
264,0.539337,0.517125,0.582467,0.641536,4.983316


In [51]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 999, … 1213]",7
1,"[1, 103, … 1814]",4
2,"[2, 1974, 582]",3
3,"[3, 1278, … 300]",5
4,"[4, 1254, … 491]",5


# String Nearest Neighbors

This might be very slow for very large vocab / column.

In [52]:
df = pl.DataFrame({
    "a":["AAAAA", "ABCABC", "AAAADDD", "ADSDSDS", "WORD"],
    "b":["AAAAT", "ABCACD", "ADSSD", "APPLES", "WORLD"] 
})

In [53]:
# Use Levenshtein to find the nearest neighbor in vocab to word in column a
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 1, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
str
"""AAAAT"""
"""ABCACD"""
"""AAAAT"""
"""ADSSD"""
"""WORLD"""


In [54]:
# Use Levenshtein to find 2 nearest neighbors
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 2, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD"", ""AAAAT""]"
"[""AAAAT"", ""ABCACD""]"
"[""ADSSD"", ""APPLES""]"
"[""WORLD"", ""ADSSD""]"


In [55]:
# Currently only Levenshtein and hamming are implemented for this
# Empty means nothing in vocab can be compared in the hamming sense with the corresponding word in a
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 2, 
        threshold = 4,
        metric = "hamming"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD""]"
[]
[]
[]


In [56]:
# You may provide a vocab like this
df.select(
    pl.col("a"),
    pds.query_similar_words(
        "a",
        vocab = ["WORLD", "AAAAA", "ABCDEFG", "ZIV", "TQQQ"],
        k = 3, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

a,similar_words_from_vocab
str,list[str]
"""AAAAA""","[""AAAAA"", ""ZIV"", ""WORLD""]"
"""ABCABC""","[""ABCDEFG"", ""AAAAA"", ""ZIV""]"
"""AAAADDD""","[""AAAAA"", ""WORLD"", ""ABCDEFG""]"
"""ADSDSDS""","[""ABCDEFG"", ""WORLD"", ""AAAAA""]"
"""WORD""","[""WORLD"", ""ZIV"", ""TQQQ""]"


# Using PDS Expressions On Series / NumPy arrays

In [57]:
df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
)
df.head()

actual,predicted,0-2,0-9
f64,f64,i32,i32
0.0,0.166273,2,9
0.0,0.068918,1,0
0.0,0.551903,2,1
1.0,0.520958,1,7
1.0,0.372835,0,1


In [58]:
pds.eval_series(
    df["0-2"], df["0-9"], # use series as args
    expr = "query_jaccard_col" # name of the pds expression
)

jaccard_col
f64
0.3


In [59]:
pds.eval_series(
    df["actual"], df["predicted"], # use series as args
    expr = "query_binary_metrics" # name of the pds expression
).unnest("binary_metrics")

precision,recall,f,average_precision,roc_auc
f64,f64,f64,f64,f64
0.496641,0.494245,0.49544,0.497287,0.498393


In [60]:
pds.eval_series(
    # can also use NumPy
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    expr = "query_psi", # name of the pds expression
    n_bins = 5, 
    return_report = True
).unnest("psi")

cnt<=,baseline_pct,actual_pct,psi_bin
f64,f64,f64,f64
0.194982,0.2,0.191,0.000414
0.403372,0.2,0.215,0.001085
0.606432,0.2,0.201,0.000005
0.81213,0.2,0.207,0.000241
inf,0.2,0.186,0.001016


In [61]:
pds.eval_series(
    np.random.random(size = 1000), 
    expr = "query_cid_ce", # name of the pds expression
)

cid_ce
f64
13.091404


In [62]:
pds.eval_series(
    pl.Series(values=np.random.random(size = 1000)), 
    expr = "query_c3_stats", # name of the pds expression
    lag = 3
)

c3_stats
f64
0.133464


# Examples using Ball Tree Features

Not currently available.

In [63]:
# size = 1_000
# df = pl.DataFrame({
#     "f": np.sin(list(range(size)))
#     , "time_idx": range(size)
#     , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
#     , "actual": np.round(np.random.random(size=size)).astype(np.int32)
#     , "predicted": np.random.random(size=size)
#     , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
# }).with_columns(
#     pds.random(0., 1.).alias("x1")
#     , pds.random(0., 1.).alias("x2")
#     , pds.random(0., 1.).alias("x3")
#     , pds.random(0., 1.).alias("a")
#     , pds.random(0., 1.).alias("b")
# ).with_columns(
#     y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001
# )
# df = df.with_columns(pl.int_range(0, size).cast(dtype=pl.UInt32).alias("index"))
# df.head()

In [64]:
# # Pointwise Nearest Neighbors
# df.with_columns(
#     pds.query_bt_knn_ptwise(
#         pl.col("x2"), 
#         pl.col("x3"), 
#         index=pl.col("index"),
#         r=999.0,
#         k =5,
#         distance_metric="haversine",
#         parallel=True).alias("ball_tree_knn_ptwise")
# )

In [65]:
# # Pointwise Nearest Neighbors with distances
# # we get a struct with an ids column and a dist column

# df.with_columns(
#     pds.query_bt_knn_ptwise(
#         pl.col("x2"), 
#         pl.col("x3"), 
#         index=pl.col("index"),
#         r=999.0,
#         k =5,
#         distance_metric="haversine",
#         return_dist=True,
#         parallel=True).alias("ball_tree_knn_ptwise")
# ).unnest("ball_tree_knn_ptwise")

In [66]:
# # Frequency Count
# df.with_columns(
#     pds.query_bt_knn_radius_freq_cnt(
#         pl.col("x2"), 
#         pl.col("x3"), 
#         index=pl.col("index"), 
#         r=999.0, 
#         k =5, 
#         distance_metric="haversine", 
#         parallel=True
#         ).alias("ball_tree_knn_radius_freq_cnt")
# )

In [67]:
# # Average distance of k nearest neighbors
# df.with_columns(
#     pds.query_bt_knn_avg(
#         pl.col("x2"), 
#         pl.col("x3"), 
#         pl.col("x1"), 
#         index=pl.col("index"), 
#         r=999.0, 
#         k =1,
#         distance_metric="euclidean", 
#         parallel=True).alias("btree_knn_avg")
# )

In [68]:
# # Count neighbors within radius
# df.with_columns(
#     pds.query_bt_nb_cnt(
#         pl.col("x2"),
#         pl.col("x3"),
#         index=pl.col("index"),
#         r=18.0, 
#         distance_metric="haversine",
#         parallel=True).alias("btree_within_radius")
# )

In [69]:
# # returns true if the row is within k nearest neighbors of the given point
# df.with_columns(
#     pds.bt_within_dist_from(
#         pl.col("x2"),
#         pl.col("x3"),
#         pt=[123,78.99],
#         r=999999, 
#         distance_metric="haversine",
#         parallel=True).alias("btree_within_dist_from")
# )

In [70]:
# # check if a point is included in the k nearest neighbors of the given point
# # Note: This is an exact match so your point could be quite close but now show up
# # We can pass in an EPSILON to account for this. THis is defaulted to the EPSILON of the data type in Rust

# # pick a random point
# point = df.select(pl.col("x2"), pl.col("x3"))
# rr = list(point.row(2))
# df.with_columns(
#     pds.is_bt_knn_from(
#         pl.col("x2"),
#         pl.col("x3"),
#         pt=rr, 
#         k=56, 
#         distance_metric="haversine",
#         parallel=True, epsilon=0.5).alias("btree_nn_within")
# )